## This notebook contains all steps used to run classifier on the precomputed scores - See notebook Submission_pipeline.ipynb for how those scores were computed


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd

import pandas as pd
train_df = pd.read_csv('submission_train.csv')
train_df.head()

,line,label,emoji_scores,fake_lables,spam_score,vadar,emotion_scores,sentiment_scores
0,<user> i dunno justin read my mention or not ....,1,0.0,0,0,0.6597,1,1
1,"because your logic is so dumb , i will not eve...",1,0.0,0,0,-0.6418,-1,-1
2,<user> just put casper in a box ! looved the b...,1,0.0,0,0,-0.7896,1,1
3,<user> <user> thanks sir > > do not trip lil m...,1,0.0,0,0,0.4926,1,1
4,visiting my brother tmr is the bestest birthda...,1,0.0,0,0,0.5826,1,1


## Swap orders cause there were differences in ways my team mate and i built the training dataframe

In [ ]:
samuels_pred_df = pd.read_csv('training_no_duplicates.csv')

In [ ]:
num_rows =1127644
# Split the DataFrame into two parts
first_part = train_df.iloc[:num_rows]
second_part = train_df.iloc[num_rows:]
# Concatenate the second part first, followed by the first part
train_df = pd.concat([second_part, first_part]).reset_index(drop=True)
train_df.rename(columns={'sentiment_scores':'sentiment_scores2'}, inplace=True)
train_df.loc[:, 'sentiment_scores'] = samuels_pred_df['predictions']

In [ ]:
train_df.loc[:, 'sentiment_scores'] = train_df.loc[:, 'sentiment_scores'].replace(0,-1)
train_df.loc[:, 'emoji_scores'] = train_df['emoji_scores'].replace(0, np.nan)
train_df.loc[:,'spam_score'] = train_df['spam_score'].replace(0,np.nan)
train_df.loc[:,'fake_lables'] = train_df['fake_lables'].replace(0,np.nan)
train_df.loc[:,'spam_score']= train_df['spam_score'].replace(1,-1)
def modify_value(x):
    if x == 0.0 or np.isnan(x):
        return np.nan
    elif x > 0.0:
        return 1
    else:
        return -1
train_df.loc[:, 'emoji_scores'] = train_df['emoji_scores'].apply(modify_value)
train_df.loc[:, 'vadar'] = train_df['vadar'].apply(modify_value)

## Null out the vadar and emotion scores to improve accuracy

In [ ]:
## Nulling out the emotion_scores
train_df.loc[train_df['emoji_scores'] == -1, 'emotion_scores'] = np.nan
train_df.loc[train_df['emoji_scores'] == -1, 'vadar'] = np.nan
train_df.loc[train_df['fake_lables'] == -1, 'emotion_scores'] = np.nan
train_df.loc[train_df['fake_lables'] == -1, 'vadar'] = np.nan
train_df.loc[train_df['spam_score'] == -1, 'emotion_scores'] = np.nan
train_df.loc[train_df['spam_score'] == -1, 'vadar'] = np.nan

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, classification_report
from sklearn.model_selection import GridSearchCV
# Define the target variable y and the features X
y = train_df['label']
X = train_df[['emoji_scores', 'spam_score', 'fake_lables', 'emotion_scores','sentiment_scores','vadar','sentiment_scores2']]

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

params = {
    'learning_rate': 0.01,
    'max_iter': 300,
    'max_leaf_nodes': 127,
    'max_depth': 5,  # Choose one of the values from [3, 5, 7]
    'min_samples_leaf': 20,
    'l2_regularization': 0,
    'early_stopping': True
}

# Initialize the model with the specified parameters
model = HistGradientBoostingClassifier(**params)
model.fit(X_train, y_train)

# Make predictions and evaluate the model
y_pred = model.predict(X_test)
y_pred_proba = model.predict_proba(X_test)[:, 1]  # Probability scores for the positive class

accuracy_sam = accuracy_score(y_test, y_pred)
precision_sam = precision_score(y_test, y_pred)
recall_sam = recall_score(y_test, y_pred)
precision_sam_neg = precision_score(y_test, y_pred,pos_label =-1)
recall_sam_neg = recall_score(y_test, y_pred,pos_label =-1)
print(f"accuracy: {accuracy_sam:.4f}")
print(f"precision_pos: {precision_sam:.4f}")
print(f"recall_pos: {recall_sam:.4f}")
print(f"precision_neg: {precision_sam_neg:.4f}")
print(f"recall_neg: {recall_sam_neg:.4f}")

accuracy: 0.8815
precision_pos: 0.8722
recall_pos: 0.8922
precision_neg: 0.8910
recall_neg: 0.8709


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, classification_report
from sklearn.model_selection import GridSearchCV
# Define the target variable y and the features X
y = train_df['label']
X = train_df[['emoji_scores', 'spam_score', 'fake_lables', 'emotion_scores','sentiment_scores']]

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

params = {
    'learning_rate': 0.01,
    'max_iter': 300,
    'max_leaf_nodes': 127,
    'max_depth': 5,  # Choose one of the values from [3, 5, 7]
    'min_samples_leaf': 20,
    'l2_regularization': 0,
    'early_stopping': True
}

# Initialize the model with the specified parameters
model = HistGradientBoostingClassifier(**params)
model.fit(X_train, y_train)

# Make predictions and evaluate the model
y_pred = model.predict(X_test)
y_pred_proba = model.predict_proba(X_test)[:, 1]  # Probability scores for the positive class

accuracy_sam = accuracy_score(y_test, y_pred)
precision_sam = precision_score(y_test, y_pred)
recall_sam = recall_score(y_test, y_pred)
precision_sam_neg = precision_score(y_test, y_pred,pos_label =-1)
recall_sam_neg = recall_score(y_test, y_pred,pos_label =-1)
print(f"accuracy: {accuracy_sam:.4f}")
print(f"precision_pos: {precision_sam:.4f}")
print(f"recall_pos: {recall_sam:.4f}")
print(f"precision_neg: {precision_sam_neg:.4f}")
print(f"recall_neg: {recall_sam_neg:.4f}")

accuracy: 0.8815
precision_pos: 0.8722
recall_pos: 0.8922
precision_neg: 0.8910
recall_neg: 0.8709


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, classification_report
from sklearn.model_selection import GridSearchCV
# Define the target variable y and the features X
y = train_df['label']
X = train_df[['emoji_scores', 'spam_score', 'fake_lables', 'emotion_scores','sentiment_scores','vadar','sentiment_scores2']]

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

params = {
    'learning_rate': 0.01,
    'max_iter': 300,
    'max_leaf_nodes': 127,
    'max_depth': 5,  # Choose one of the values from [3, 5, 7]
    'min_samples_leaf': 20,
    'l2_regularization': 0,
    'early_stopping': True
}

# Initialize the model with the specified parameters
model = HistGradientBoostingClassifier(**params)
model.fit(X_train, y_train)

# Make predictions and evaluate the model
y_pred = model.predict(X_test)
y_pred_proba = model.predict_proba(X_test)[:, 1]  # Probability scores for the positive class

accuracy_sam = accuracy_score(y_test, y_pred)
precision_sam = precision_score(y_test, y_pred)
recall_sam = recall_score(y_test, y_pred)
precision_sam_neg = precision_score(y_test, y_pred,pos_label =-1)
recall_sam_neg = recall_score(y_test, y_pred,pos_label =-1)
print(f"accuracy: {accuracy_sam:.4f}")
print(f"precision_pos: {precision_sam:.4f}")
print(f"recall_pos: {recall_sam:.4f}")
print(f"precision_neg: {precision_sam_neg:.4f}")
print(f"recall_neg: {recall_sam_neg:.4f}")

accuracy: 0.8817
precision_pos: 0.8733
recall_pos: 0.8917
precision_neg: 0.8904
recall_neg: 0.8717


In [ ]:
X_test.head()

,emoji_scores,spam_score,fake_lables,emotion_scores,sentiment_scores,vadar,sentiment_scores2
769240,NaN,NaN,NaN,-1.0,-1,-1.0,-1
1065138,NaN,NaN,NaN,-1.0,-1,-1.0,-1
170717,NaN,NaN,NaN,-1.0,-1,-1.0,-1
1399111,NaN,NaN,NaN,1.0,1,1.0,1
101588,-1.0,NaN,-1.0,NaN,-1,NaN,1


In [ ]:
only_sent = X_test['sentiment_scores']
accuracy_sam = accuracy_score(y_test, only_sent)
precision_sam = precision_score(y_test, only_sent)
recall_sam = recall_score(y_test, only_sent)
precision_sam_neg = precision_score(y_test, only_sent,pos_label =-1)
recall_sam_neg = recall_score(y_test, only_sent,pos_label =-1)
print(f"accuracy: {accuracy_sam:.4f}")
print(f"precision_pos: {precision_sam:.4f}")
print(f"recall_pos: {recall_sam:.4f}")
print(f"precision_neg: {precision_sam_neg:.4f}")
print(f"recall_neg: {recall_sam_neg:.4f}")

accuracy: 0.8817
precision_pos: 0.8733
recall_pos: 0.8917
precision_neg: 0.8904
recall_neg: 0.8717
